# Análise Titanic (Parte 3)- Escolha dos Hiperparâmetros


Neste notebook iremos continuar com a análise de dados dos passageiros do Titanic. Agora iremos trabalhar com a escolha dos hiperparâmetros dos modelos.


## Etapa inicial - Baixar os dados e tratá-los

In [1]:
import pandas as pd

## Baixar os dados
dados = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')

## Preencher idades faltantes
dados.isna().sum()

## Inserir Idades faltantes com a Mediana
dados['age'].fillna(dados['age'].median(), inplace = True)

## Criar variável "familia" como a soma entre 'sibsp' e 'parch'
dados['familia'] = dados['sibsp'] + dados['parch'] 

## Transformar variáveis categóricas
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit_transform(dados['sex'])

dados_ajustados2 = pd.get_dummies(dados, columns=["pclass","sex"])

del dados_ajustados2['sex_female']
del dados_ajustados2['pclass_1']

resultados_acuracia2 = pd.DataFrame(columns = ['modelo', 'acuracia'])

#### Padronizar as variáveis quantitativas
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(dados_ajustados2[['age', 'familia']])
dados_ajustados2[['age', 'familia']] = std_scale.transform(dados_ajustados2[['age','familia']])

## Exemplo de escolha dos hiperparâmetros do KNN

O modelo KNN tem como hiperparâmetro o número de "vizinhos" (neighbors) que é definido no momento que criamos o 'objeto' do sklearn. Por exemplo, ao criarmos uma instância do knn que utilize os 3 vizinhos mais próximos:

In [13]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

Para escolher o melhor hiperparâmetro "vizinhos" iremos utilizar o que é chamado de Validação Cruzada (Cross Validation)


## Cross Validation

A ideia é separar a base de dados em grupos de treinamento e grupos de teste. O modelo a ser avaliado é ajustado com os dados presentes no grupo de treinamento e, depois, a performance do modelo ajustado é avaliada com os dados presentes no grupo de teste. A performance do modelo é a média dos resultados obtidos em cada rodada de treinamento.
Existem diversas maneiras de fazer a separação entre dados de treinamento e teste. 
Dentre elas, temos:

### K-Fold

Dividiremos a base de dados em K partições ('folds') e realizaremos K rodadas de treinamento e teste do modelo com essas combinações de dados. 

In [7]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0041.png")


### Shuffle Split

Iremos dividir a base em um número definido de divisões treino/teste. As amostras são embaralhadas e depois divididas nos pares de treino/teste


In [11]:
Image(url= "https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0061.png")

Outras metodologias de Cross Validation, como Leave One Out Cross Validation (LOOCV), estão descritas em

https://scikit-learn.org/stable/modules/cross_validation.html

Para encontrar o melhor hiperparâmetro do KNN, iremos buscar qual tem melhor desepenho em um processo de Cross Validation.
O Scikit Learn permite realizar essa busca de duas maneiras: GridSearch (onde todos os hiperparâmetros serão testados) e RandomizedSearch (onde os hiperparâmetros serão buscados de maneira aleatória). Este último para casos onde a quantidade de hiperparâmetros inviabilizaria o cálculo de todas as combinações possíveis.



In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit

X = dados_ajustados2[['pclass_2','pclass_3','age','sex_male','familia']]
y = dados_ajustados2['survived']

# KNN a ser utilizado
knn = KNeighborsClassifier()

# Todos os parâmetros a serem testados
parameters = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}

# Cross Validation com o Shuffle Split
cross_validation = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

best_knn = GridSearchCV(knn, parameters, cv= cross_validation)
best_knn.fit(X,y)

# Qual melhor hiperparâmetro
best_knn.best_params_

{'n_neighbors': 19}

No caso, o melhor hiperparâmetros para o KNN é de 19

## Busca dos melhores hiperparâmetros para SVM

In [15]:
from sklearn.model_selection import KFold
from sklearn import svm

kf = KFold(n_splits=10)

parameters = {'kernel':('linear', 'rbf'), 'C':[1,2,3,4,5,6,7,8,9,10], 'degree': [2,3,4,5]}

svc = svm.SVC()

best_svm = GridSearchCV(svc, parameters, cv=kf)
best_svm.fit(X, y)

best_svm.best_params_

{'C': 2, 'degree': 2, 'kernel': 'rbf'}